In [1]:
# Importing the required library
import sys
import os 
import gym
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import random
import math
from torch.utils.tensorboard import SummaryWriter
import time 
import argparse
sys.argv = ['']
del sys



c:\Users\kt471\Anaconda3\envs\clean_rl\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
c:\Users\kt471\Anaconda3\envs\clean_rl\lib\site-packages\botocore\httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext


In [2]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("-v_1", "--verbose_1", help=" ",
                    action="store_true", default='Original+linear layer to see if Q is negative ' )
parser.add_argument("-seed", "--seed", help="increase output verbosity",
                    action="store_true",default = 1)
parser.add_argument("-NUMBER_OF_EPISODES", "--NUMBER_OF_EPISODES", help="Total_Episodes to run ",
                    action="store_true",default = 200)
parser.add_argument("-MAX_STEPS", "--MAX_STEPS", help="MAX_STEPS to run each episode ",
                    action="store_true",default = 1369)
parser.add_argument("-LEARNING_RATE", "--LEARNING_RATE", help="LEARNING_RATE ",
                    action="store_true",default = 0.0001) #0.0001

parser.add_argument("-UCB_LEARNING_RATE", "--UCB_LEARNING_RATE", help="UCB_LEARNING_RATE ",
                    action="store_true",default = 0.0001) #0.0001

parser.add_argument("-UCB_FILTER", "--UCB_FILTER", help="UCB_FILTER ",
                    action="store_true",default = 0.0001)

parser.add_argument("-DISCOUNT_FACTOR", "--DISCOUNT_FACTOR", help="DISCOUNT_FACTOR ",
                    action="store_true",default = 0.99)

parser.add_argument("-HIDDEN_LAYER_SIZE", "--HIDDEN_LAYER_SIZE", help="HIDDEN_LAYER_SIZE ", 
                    action="store_true",default = 128)

parser.add_argument("-UCB_HIDDEN_LAYER_SIZE", "--UCB_HIDDEN_LAYER_SIZE", help="UCB HIDDEN_LAYER_SIZE ",
                    action="store_true", default=128)

parser.add_argument("-EGREEDY_START", "--EGREEDY_START", help="EGREEDY_START ",
                    action="store_true",default = 0.4) #0.4
parser.add_argument("-EGREEDY_FINAL", "--EGREEDY_FINAL", help="EGREEDY_FINAL ",
                    action="store_true",default = 0.01) #0.01
parser.add_argument("-EGREEDY_DECAY", "--EGREEDY_DECAY", help="EGREEDY_DECAY- not being used ",
                    action="store_true",default = 2000)
parser.add_argument("-REPLAY_BUFFER_SIZE", "--REPLAY_BUFFER_SIZE", help="REPLAY_BUFFER_SIZE ",
                    action="store_true",default = 40000)
parser.add_argument("-BATCH_SIZE", "--BATCH_SIZE", help="BATCH_SIZE ",
                    action="store_true",default = 64)
parser.add_argument("-UPDATE_TARGET_FREQUENCY", "--UPDATE_TARGET_FREQUENCY", help="UPDATE_TARGET_FREQUENCY ",
                    action="store_true", default=2000)
parser.add_argument("-TEST_MODEL_FREQ", "--TEST_MODEL_FREQ", type=int, help="After How many episodes you want to test the model ",
                     default=1)

parser.add_argument("-TEST_RUN_EPISODE", "-TEST_RUN_EPISODE", help=" Number of episodes to run while testing . ",
                    action="store_true", default=10)


parser.add_argument("-MODEL_SAVE_FREQUENCY", "--MODEL_SAVE_FREQUENCY", help="After How many episodes you want to save the model ",
                    action="store_true", default=10)

parser.add_argument("-TRAINING_STARTS", "--TRAINING_STARTS", type=int, help="After How many episodes does the optimization starts,currently 10 cycles. ",
                     default=0)

parser.add_argument("-TAU", "--TAU", type=int, help="Target Network Update.,tau = 0 no update , tau = 1 imediate update of target with latest, new_target = q * tau + (1-tau) * target ",
                     default=0.5)




parser.add_argument("-run_name", "--run_name", help="run name",
                    action="store_true", default=f"{'_vehcile_13_random_initial_SOC'}_{int(time.time())}")
args = parser.parse_args()
# args = argparse.Namespace(verbose=False, verbose_1=False)


In [3]:
args.TEST_MODEL_FREQ

1

In [4]:
writer = SummaryWriter(f"runs/{'DDQN'}{args.run_name}")


In [5]:
# selecting the available device (cpu/gpu)

use_cuda = torch.cuda.is_available()

device = torch.device("cuda:0" if use_cuda else "cpu")

print(device)

cpu


In [6]:
# Creating the CartPole-v0 environment

# env = gym.make('CartPole-v0')
env = gym.make('Vehicle-v0') 
env.reset()
# Creating the CartPole-v0 environment

# env = gym.make('CartPole-v0')
# env = gym.make('Vehicle-fixedSOC')
env.reset()
action = env.action_space.sample()
new_state, reward, done, info = env.step(action)
# new_state[2]
print(action)
new_state[2]


initial soc is 0.3
331


-0.40000919697613346

In [7]:
# setting the seed value for reproducibility

# seed = 1626
env.seed(args.seed)
torch.manual_seed(args.seed)
random.seed(args.seed)


In [8]:
# run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
# run_name = f"{seed}__{int(time.time())}"


In [9]:
# Fetching the number of states and actions
#number_of_states = env.observation_space.n
number_of_states = env.observation_space.shape[0]
number_of_actions = env.action_space.n
# checking the total number of states and action
print('Total number of States : {}'.format(number_of_states)) 
print('Total number of Actions : {}'.format(number_of_actions))

Total number of States : 3
Total number of Actions : 2000


In [10]:
# model = nn.Sequential(
#     nn.Linear(1, 1, bias=False),
#     nn.Sigmoid(),
#     nn.Linear(1, 1, bias=False)
# )

# criterion = nn.MSELoss()
# x = torch.randn(1, 1)
# target = torch.ones(1, 1)

# output = model(x)
# loss = criterion(output, target)
# loss.backward()

# lin1_grad = model[0].weight.grad.clone()
# lin2_grad = model[2].weight.grad.clone()
# print('Before regularization')
# print('Grad lin1: {}'.format(lin1_grad))
# print('Grad lin2: {}'.format(lin2_grad))

# # Add regularization to lin1
# model.zero_grad()
# output = model(x)
# loss = criterion(output, target)
# loss = loss + torch.norm(model[0].weight)
# loss.backward()

# lin1_grad_reg = model[0].weight.grad.clone()
# lin2_grad_reg = model[2].weight.grad.clone()
# print('After regularization')
# print('Grad lin1: {}'.format(lin1_grad_reg))
# print('Grad lin2: {}'.format(lin2_grad_reg))
# print('As you can see, the gradient for lin2 stays the same, while the gradient for lin1 changes.This is due to the fact, that the parameters of lin2 were not involved in creating the regularization term, so they won’t be touched.')


In [11]:
# # Setting the Hyper parameter Values for Q Learning

# NUMBER_OF_EPISODES = 4000
# MAX_STEPS = 1369
# LEARNING_RATE = 0.001
# DISCOUNT_FACTOR = 0.99
# HIDDEN_LAYER_SIZE = 128

# EGREEDY_START = 0.9
# EGREEDY_FINAL = 0.002
# EGREEDY_DECAY = 2200

# REPLAY_BUFFER_SIZE = 20000
# BATCH_SIZE = 64

# UPDATE_TARGET_FREQUENCY = 1000

In [12]:
def linear_schedule(EGREEDY: float, EGREEDY_FINAL: float, duration: int, t: int):
    slope = (EGREEDY_FINAL - EGREEDY) / duration
    return max(slope * t + EGREEDY, EGREEDY_FINAL)


In [13]:
# def calculate_epsilon(steps_done):
#     """
#     Decays eplison with increasing steps
#     Parameter:
#     steps_done (int) : number of steps completed
#     Returns:
#     int - decayed epsilon
#     """
#     epsilon = EGREEDY_FINAL + (EGREEDY - EGREEDY_FINAL) * \
#               math.exp(-1. * steps_done / EGREEDY_DECAY )
#     return epsilon

In [14]:
# Deep Q Network Model Architecture

class DQN(nn.Module):
    def __init__(self , hidden_layer_size):
        super().__init__()
        torch.manual_seed(args.seed)
        self.hidden_layer_size = hidden_layer_size
        # self.fc0 = nn.Linear(number_of_states,self.hidden_layer_size)
        self.fc1 = nn.Linear(number_of_states, self.hidden_layer_size)
        # self.fc2 = nn.Linear( self.hidden_layer_size, self.hidden_layer_size)
        # self.fc3 = nn.Linear( self.hidden_layer_size, self.hidden_layer_size)
        # self.fc4 = nn.Linear( self.hidden_layer_size, self.hidden_layer_size)
        self.fc2 = nn.Linear(self.hidden_layer_size,number_of_actions)
        
    # def _init_weights(self, module):
    #     if isinstance(module, nn.Linear):
    #         module.weight.data.normal_(mean=0.0, std=config.initializer_range)
    #         if module.bias is not None:
    #             module.bias.data.zero_()

    def forward(self, x):
        # x = self.fc0(x)
        output = torch.relu(self.fc1(x))
        # output = self.fc1(x)
        # output = torch.relu(self.fc2(output))
        # output = torch.relu(self.fc3(output))
        # output = torch.relu(self.fc4(output))
        output = self.fc2(output)
        return output

In [15]:
np.mean([-100,180])

40.0

In [16]:
def minmaxnorm(x,Min,Max):
    meanX = np.mean([Min,Max])
    x_out = 2*(x - meanX)/(Max-Min)
    return(x_out)
    

In [17]:
minmaxnorm(0,0,1)

-1.0

In [18]:
class ExperienceReplay(object):
    def __init__(self , capacity):
        
        self.capacity = capacity
        self.buffer = []
        self.pointer = 0
    
    def push(self , state, action, new_state, reward, done):
        experience = (state, action, new_state, reward, done)
        
        if self.pointer >= len(self.buffer):
            self.buffer.append(experience)
        else:
            self.buffer[self.pointer] = experience
        
        self.pointer = (self.pointer + 1) % self.capacity
        
    def sample(self , batch_size):
        # random.seed(args.seed)
        return zip(*random.sample(self.buffer , batch_size))
    
    def __len__(self):
        return len(self.buffer)    

In [19]:
# Instantiating the ExperienceReplay
memory = ExperienceReplay(args.REPLAY_BUFFER_SIZE)


In [20]:
# Building the brain of the network i.e. the DQN Agent

class DQN_Agent(object):
    def __init__(self):
        torch.manual_seed(args.seed)
        self.dqn = DQN(args.HIDDEN_LAYER_SIZE).to(device)
        self.target_dqn = DQN(args.HIDDEN_LAYER_SIZE).to(device)
        self.target_dqn.load_state_dict(self.dqn.state_dict())
        
        # self.criterion = torch.nn.MSELoss()
        self.criterion = torch.nn.SmoothL1Loss(size_average=None, reduce=None, reduction='mean', beta=5.0)
        
        self.optimizer = optim.Adam(
            params=self.dqn.parameters(), lr=args.LEARNING_RATE)
        
        self.target_dqn_update_counter = 0
        self.best_reward = -99999
        self.prev_best_episode = 0
    
    def select_action(self, state, EGREEDY):
        
        random_for_egreedy = torch.rand(1)[0]
        
        if random_for_egreedy > EGREEDY:      
            
            with torch.no_grad():
                
                state = torch.Tensor(state).to(device)
                q_values = self.dqn(state) # + round(random.uniform(0.1, 5), 1)
                action = torch.max(q_values,0)[1]
                action = action.item()        
        else:
            # random.seed(args.seed)
            action = env.action_space.sample()
            # action = 1998
            # print(action)
            
        # print(action)
        return action
    
    def optimize(self,epi,LR):
        self.optimizer = optim.Adam(
            params=self.dqn.parameters(), lr=LR)
        
        if (args.BATCH_SIZE > len(memory)):
            return
        
        state, action, new_state, reward, done = memory.sample(args.BATCH_SIZE)
        # print(reward)
        
        state = torch.Tensor(state).to(device)
        new_state = torch.Tensor(new_state).to(device)
        reward = torch.Tensor(reward).to(device)
        action = torch.LongTensor(action).to(device)
        done = torch.Tensor(done).to(device)
        
        # select action : get the index associated with max q value from prediction network
        new_state_indxs = self.dqn(new_state).detach() 
        max_new_state_indxs = torch.max(new_state_indxs, 1)[1] # to get the max new state indexes
        # print(max_new_state_indxs)
        
        
        # Using the best action from the prediction nn get the max new state value in target dqn
        new_state_values = self.target_dqn(new_state).detach()
        max_new_state_values = new_state_values.gather(1, max_new_state_indxs.unsqueeze(1)).squeeze(1)
        # print(max_new_state_values)
        
        target_value = reward + (1 - done) * args.DISCOUNT_FACTOR * max_new_state_values #when done = 1 then target = reward
        
        predicted_value = self.dqn(state).gather(1, action.unsqueeze(1)).squeeze(1)
        # print()
        # print(predicted_value,target_value)
        loss = self.criterion(predicted_value, target_value)
        # loss = self.criterion(target_value, predicted_value)
        loss_output = self.criterion(predicted_value, target_value)
        # loss_output = self.criterion(target_value, predicted_value)
        loss_output_diff = predicted_value - target_value
        # loss_output_diff = target_value - predicted_value
        # print('loss',loss.mean().item())
        writer.add_scalar("train/losses/td_loss_steps_abs", loss_output_diff.mean().item(), epi)
        writer.add_scalar("train/losses/td_loss_steps", loss.mean().item(), epi)
        writer.add_scalar("train/losses/predictedmean", predicted_value.mean().item(), epi)
        writer.add_scalar("train/losses/targetmean", target_value.mean().item(), epi)
        writer.add_scalar("train/losses/predictedmin", predicted_value.min().item(), epi)
        writer.add_scalar("train/losses/targetmin", target_value.min().item(), epi)
        writer.add_scalar("train/losses/predictedmax",
                          predicted_value.max().item(), epi)
        writer.add_scalar("train/losses/targetmax", target_value.max().item(), epi)
    
        self.optimizer.zero_grad()
        loss.backward()
        # for param in self.dqn.parameters():
        #     param.grad.data.clamp_(-1, 1)
        for tag, value in self.dqn.named_parameters():
            writer.add_scalar("train/charts/gradsum ",
                            np.array(value.grad.cpu()).sum(), epi)
            writer.add_scalar("train/charts/gradmean", np.array(value.grad.cpu()).mean(), epi)
            writer.add_scalar("train/charts/gradmin", np.array(value.grad.cpu()).min(), epi)
            writer.add_scalar("train/charts/gradmax", np.array(value.grad.cpu()).max(), epi)
        self.optimizer.step()
        # torch.nn.utils.clip_grad_norm(
        #     parameters=self.dqn.parameters(), max_norm=1, norm_type=2.0)
        
        if (self.target_dqn_update_counter % args.UPDATE_TARGET_FREQUENCY == 0):
            # self.target_dqn.load_state_dict(self.dqn.state_dict())
            ## new
            target_net_state_dict = self.target_dqn.state_dict()
            policy_net_state_dict = self.dqn.state_dict()
            for key in policy_net_state_dict:
                target_net_state_dict[key] = policy_net_state_dict[key] * \
                args.TAU + target_net_state_dict[key]*(1-args.TAU)
            self.target_dqn.load_state_dict(target_net_state_dict)

        #     for target_network_param, q_network_param in zip(self.target_dqn.parameters(), self.dqn.parameters()):
        #         target_network_param.data.copy_(
        #             0.5 * q_network_param.data +
        #             (1.0 - 0.5) * target_network_param.data
        #         )
        #     self.target_dqn.load_state_dict(target_network_param.data.state_dict())
        # self.target_dqn_update_counter += 1

            
        return (loss.mean().item())
        
    def save_model(self,epi,total_epi):
        if (epi % args.MODEL_SAVE_FREQUENCY ==0):
            torch.save(self.dqn.state_dict(),
                       f"runs/{'DDQN'}{args.run_name}/{'DDQN'}{args.run_name}_{'episode#'}_{epi}.pt")
            
    def log_gradients_in_model(model, logger, step):
        for tag, value in model.named_parameters():
            if value.grad is not None:
                self.logger.add_histogram(tag + "/grad", value.grad.cpu(), step)
                
    def test_ddqn(self,epi):
        print('###############Running Test#############################')
        train_steps_counter = 0
        env = gym.make('Vehicle-fixedSOC')
        # env = gym.make('Vehicle-v0')
        obs = env.reset()
        reward_total_ddqn = []
        reward_app_ddqn = []
        fuel_total_ddqn = []
        fuel_app_ddqn = []
        a1 = []
        s1 = []
        s2 = []
        s3 = []
        train_steps_counter_all = []
        if (epi % args.TEST_MODEL_FREQ ==0):
            
            for i in range(args.MAX_STEPS):
                torch.manual_seed(args.seed)
                model = DQN(args.HIDDEN_LAYER_SIZE).to(device)
                model.load_state_dict(self.dqn.state_dict())
                q_values = model(torch.Tensor(obs).to(device))
                if(epi < 2):
                    actions = 1000
                else:
                    actions = torch.argmax(q_values).cpu().numpy()
                    # print('q_values:', np.max(q_values.cpu().detach().numpy()))
                    q_max = np.max(q_values.cpu().detach().numpy())
                    writer.add_scalar("test/q_max", q_max,
                                      train_steps_counter)
                # print(actions)
                next_obs, rewards, dones, infos = env.step(actions)
                obs = next_obs
                [state1,state2,state3] = obs
                fuel_kg = infos[48]
                writer.add_scalar("test/state1", state1, train_steps_counter)
                writer.add_scalar("test/state2", state2, train_steps_counter)
                writer.add_scalar("test/state3", state3, train_steps_counter)
                writer.add_scalar("test/reward", rewards, train_steps_counter)
                writer.add_scalar("test/action", actions, train_steps_counter)
                
                train_steps_counter += 1
                reward_app_ddqn.append(float(rewards))
                fuel_app_ddqn.append(float(fuel_kg))
                # a1.append(float(actions))
                # s1.append(float(state1))
                # s2.append(float(state2))
                # s3.append(float(state3))
                train_steps_counter_all.append(train_steps_counter)
                if dones:
                    reward_total_ddqn = sum(reward_app_ddqn)
                    fuel_total_ddqn = sum(fuel_app_ddqn)/len(fuel_app_ddqn)
                    writer.add_scalar("test/reward_total", reward_total_ddqn, epi)
                    writer.add_scalar("test/fuel_total_ddqn",
                                      fuel_total_ddqn, epi)
                    # self.best_reward = max(self.best_reward , reward_total)
                    if(epi < 1):
                        self.best_reward = reward_total_ddqn
                        self.prev_best_episode = 0
                        
                    if(self.best_reward > reward_total_ddqn):
                        self.best_reward = self.best_reward
                        self.prev_best_episode = self.prev_best_episode
                        print('###Best reward not changed ,best reward , prev episode###',self.best_reward,self.prev_best_episode)
                    else:
                        self.best_reward = reward_total_ddqn
                        self.prev_best_episode = epi
                        print('###Best reward new episode and best new episode is### ',
                              self.best_reward, self.prev_best_episode)
                        torch.save(self.dqn.state_dict(),
                                   f"runs/{'DDQN'}{args.run_name}/{'DDQN'}{args.run_name}_{'best_reward episode#'}_{epi}.pt")
                        # writer.add_scalar("best/s1", s1, train_steps_counter_all)
                        # writer.add_scalar("best/s2", s2, train_steps_counter_all)
                        # writer.add_scalar("best/s3", s3, train_steps_counter_all)
                        # writer.add_scalar("best/a1", a1, train_steps_counter_all)
                        
                        
            
            
        
        return
        

In [21]:
# modeltest = DQN_Agent()
# modeltest.dqn.named_parameters()
# for name, param in modeltest.dqn.named_parameters():
#     if param.requires_grad:
#         print (name, param.data)

In [22]:
# Instantiating the DQN Agent
torch.manual_seed(args.seed)
dqn_agent = DQN_Agent()
# torch.manual_seed(args.seed)
model_dqn = dqn_agent.dqn
# writer = SummaryWriter(f"runs/{'DDQN'}{run_name}")

In [23]:
# class UCB_NN(nn.Module):
#     def __init__(self, hidden_layer_size):
#         super().__init__()
#         torch.manual_seed(args.seed)
#         self.hidden_layer_size = hidden_layer_size
#         # self.fc0 = nn.Linear(number_of_states,self.hidden_layer_size)
#         self.fc1 = nn.Linear(1, self.hidden_layer_size)
#         # self.fc2 = nn.Linear( self.hidden_layer_size, self.hidden_layer_size)
#         # self.fc3 = nn.Linear( self.hidden_layer_size, self.hidden_layer_size)
#         # self.fc4 = nn.Linear( self.hidden_layer_size, self.hidden_layer_size)
#         self.fc2 = nn.Linear(self.hidden_layer_size, 2)

#     def forward(self, x):
#         # x = self.fc0(x)
#         output = torch.relu(self.fc1(x))
#         # output = torch.relu(self.fc2(output))
#         # output = torch.relu(self.fc3(output))
#         # output = torch.relu(self.fc4(output))
#         output = self.fc2(output)
#         return output


In [24]:
# # Define UCB class
# # Building the brain of the network i.e. the DQN Agent
# class UCB(object):
#     def __init__(self):
#         torch.manual_seed(args.seed)
#         self.ucb = UCB_NN(args.UCB_HIDDEN_LAYER_SIZE).to(device)

#         # self.criterion = torch.nn.MSELoss()
#         self.criterion = torch.nn.SmoothL1Loss()

#         self.optimizer = optim.Adam(params=self.ucb.parameters(), lr=args.UCB_LEARNING_RATE)

#     def UCB_calc(self,actions, Q_learned, states, epi):
#         self.optimizer = optim.Adam(params=self.ucb.parameters(), lr=args.UCB_LEARNING_RATE)

#         actions = torch.Tensor(actions).to(device)
#         Q_learned = torch.Tensor(Q_learned).to(device)
#         Q_current = self.ucb(actions)
#         Q_current = torch.Tensor(Q_current).to(device)
#         loss_ucb = (1-args.UCB_FILTER) * (Q_learned - Q_current)
#         loss_ucb = self.criterion(Q_learned, Q_current)
#         self.optimizer.zero_grad()
#         loss_ucb.backward()

#         # for tag, value in self.ucb.named_parameters():
#         #     writer.add_scalar("ucb/charts/gradsum ",
#         #                       np.array(value.grad.cpu()).sum(), epi)
#         #     writer.add_scalar("ucb/charts/gradmean",
#         #                       np.array(value.grad.cpu()).mean(), epi)
#         #     writer.add_scalar("ucb/charts/gradmin",
#         #                       np.array(value.grad.cpu()).min(), epi)
#         #     writer.add_scalar("ucb/charts/gradmax",
#         #                       np.array(value.grad.cpu()).max(), epi)

#         return(Q_current)


In [25]:
torch.tensor([[1, 2, 3], [4, 5, 6]])

tensor([[1, 2, 3],
        [4, 5, 6]])

In [26]:
# Instantiating the UCB class Agent
torch.manual_seed(args.seed)
# ucb_agent = UCB()

In [27]:
# ucb_agent.UCB_calc([1], 1, 0.0, 2)


In [28]:
# Write the hyperparameters to tensorboard
writer.add_text(
    "hyperparameters",
    "|param|value|\n|-|-|\n%s" % (
        "\n".join([f"|{key}|{value}|" for key, value in vars(args).items()]))
)


In [29]:
# for tag, value in model_dqn.named_parameters():
#     print(value.grad.cpu())
# # print(writer)


In [30]:
# e_counter = 1
# for episode in range(NUMBER_OF_EPISODES+1):
#     EGREEDY = linear_schedule(0.9, 0.02, NUMBER_OF_EPISODES, e_counter)
#     e_counter = e_counter +1
#     print(e_counter)
#     print(EGREEDY)
    

In [31]:
import numpy as np
steps_total = []
reward_total_all_episodes = []
steps_counter = 0
episode_counter = []
e_counter = 1
episode_done = False
loss_step_total = []
q_pred_step_total=[]
q_targ_step_total=[]
for episode in range(args.NUMBER_OF_EPISODES+1):
    print('###############Running Trainning#############################')
    env = gym.make('Vehicle-v0') 
    
    episode_counter.append(episode)
    if episode == args.NUMBER_OF_EPISODES:
        episode_done = True
        
    state = env.reset()
    done = False
    # print('episode :',e_counter)
    LR_comp = linear_schedule(
        args.LEARNING_RATE, args.LEARNING_RATE/20, args.NUMBER_OF_EPISODES, e_counter)
    writer.add_scalar("train/learning_rate", LR_comp, episode)

    step = 0
    reward_total = []
    reward_app = []
    # EGREEDY = calculate_epsilon(NUMBER_OF_EPISODES)
    if episode<args.NUMBER_OF_EPISODES-10:
        EGREEDY = linear_schedule(args.EGREEDY_START, args.EGREEDY_FINAL, args.NUMBER_OF_EPISODES, e_counter)
        # LR_comp = linear_schedule(
        #     args.LEARNING_RATE, 0.0001, args.NUMBER_OF_EPISODES, e_counter)
    else:
        EGREEDY = linear_schedule(
            args.EGREEDY_START, args.EGREEDY_FINAL, args.NUMBER_OF_EPISODES, e_counter)
        # LR_comp = linear_schedule(
        #     args.LEARNING_RATE, 0.0001, args.NUMBER_OF_EPISODES, e_counter)
    print('Epsilon:', EGREEDY)
    print('Episode:',e_counter)
   
    for i in range(args.MAX_STEPS):
        step += 1
        steps_counter += 1
        
        [state1,state2,state3] = state
        writer.add_scalar("train/states/state1", state1, steps_counter)
        writer.add_scalar("train/states/state2", state2, steps_counter)
        writer.add_scalar("train/states/state3", state3, steps_counter)
        
        writer.add_scalar("train/state1", state1, step)
        writer.add_scalar("train/state2", state2, step)
        writer.add_scalar("train/state3", state3, step)
        
        
        # if(state3 > -0.6):
        #     action = 1999
        # else:
        #     action = dqn_agent.select_action(state, EGREEDY)
        
        # if(state3 > -0.6):
        #     action = 1999
        # else:
        #     action = dqn_agent.select_action(state, EGREEDY)
            
        action = dqn_agent.select_action(state, EGREEDY)
            
        writer.add_scalar("train/action/action", action, steps_counter)
            
        new_state, reward, done, info = env.step(action)
        memory.push(state, action, new_state, reward, done)
        reward_app.append(reward)
        if(episode > args.TRAINING_STARTS):
            loss_step = dqn_agent.optimize(steps_counter,LR_comp)
            if loss_step == None:
                loss_step = 0
                q_pred_step = 0
                q_targ_step = 0
            loss_step_total.append(loss_step)
            # q_pred_step_total.append(q_pred_step)
            # q_targ_step_total.append(q_targ_step)
            state = new_state
            # os.system('cls')
            # UU = ucb_agent.UCB_calc(1, 1.0, state, episode)
            writer.add_scalar("train/charts/epsilon", EGREEDY, episode)
            if done:
                steps_total.append(step)
                reward_total = sum(reward_app)
                writer.add_scalar("train/reward", reward_total, episode)
                reward_total_all_episodes.append(reward_total)
                print('loss',sum(loss_step_total)/len(loss_step_total))
                writer.add_scalar("train/losses/td_loss",sum(loss_step_total)/len(loss_step_total), episode)
                # writer.add_scalar("losses/predicted", sum(q_pred_step_total)/len(q_pred_step_total), episode)
                # writer.add_scalar("losses/target", sum(q_targ_step_total)/len(q_targ_step_total), episode)
                # for tag, value in model_dqn.named_parameters():
                #     writer.add_scalar("charts/gradsum ", np.array(value.grad.cpu()).sum(), episode)
                #     writer.add_scalar("charts/gradmean", np.array(value.grad.cpu()).mean(), episode)
                #     writer.add_scalar("charts/gradmin", np.array(value.grad.cpu()).min(), episode)
                #     writer.add_scalar("charts/gradmax", np.array(value.grad.cpu()).max(), episode) 
                e_counter = e_counter+1
                dqn_agent.save_model(episode, args.NUMBER_OF_EPISODES)
                # writer.add_scalar("charts/episodic_return",
                #                       info["episode"]["r"], episode)
                #     # writer.add_scalar("charts/episodic_length",
                                    #   info["episode"]["l"], global_step)
                dqn_agent.test_ddqn(episode)
                break
            # if episode_done:
            

###############Running Trainning#############################
initial soc is 0.2
Epsilon: 0.39805
Episode: 1
self.steps,self.mpge,self.soc 1370 34.56826405002161 -220.5139117683313 0.16041012910096317
###############Running Trainning#############################
initial soc is 0.8
Epsilon: 0.39805
Episode: 1
self.steps,self.mpge,self.soc 1370 41.0057614410885 -163.7242435839637 0.6700913480134553
loss 0.06944615356596914
###############Running Test#############################
initial soc is 0.6
self.steps,self.mpge,self.soc 1370 27.40748792156098 -312.903232686338 0.6717904364759404
###Best reward new episode and best new episode is###  -312.903232686338 1
###############Running Trainning#############################
initial soc is 0.4
Epsilon: 0.3961
Episode: 2
self.steps,self.mpge,self.soc 1370 44.36086436115212 -142.360909013474 0.24491083756842028
loss 0.05949144079530213
###############Running Test#############################
initial soc is 0.6
self.steps,self.mpge,self.soc 1370

In [ ]:
print("Average reward: %.2f" % (sum(reward_total_all_episodes)/NUMBER_OF_EPISODES))
# print("Average reward (last 100 episodes): %.2f" % (sum(steps_total[-100:])/100))

In [ ]:
# print("Average reward: %.2f" % (sum(steps_total)/NUMBER_OF_EPISODES))
# print("Average reward (last 100 episodes): %.2f" % (sum(steps_total[-100:])/100))

In [ ]:
def moving_average_filter(panda_series, window_size):
    numbers_series = pd.Series(panda_series)
    windows = numbers_series.rolling(window_size)
    moving_averages = windows.mean()
    moving_averages_list = moving_averages.tolist()
    # mvoing_average_series = list(moving_averages_list[window_size - 1:])
    # mvoing_average_series[:1369] = [0]
    # mvoing_average_series[:1370] = [0]
    return(moving_averages_list)

In [ ]:
import pandas as pd 
import numpy as np
df_ck = pd.DataFrame()
df_ck['x'] = np.array(reward_total_all_episodes)
df_ck['moving'] = moving_average_filter(df_ck.x.values, 10)

In [ ]:
import numpy as np
import plotly.graph_objects as go
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=episode_counter, y=reward_total_all_episodes,
                         mode='lines',
                         name='regular'))
fig.add_trace(go.Scatter(x=episode_counter, y=df_ck.moving,
                         mode='lines',
                         name='regular'))

In [ ]:
# plt.figure(figsize=(12,5))
# plt.title("Rewards Collected")
# plt.xlabel('Steps')
# plt.ylabel('Reward')
# plt.bar(np.arange(len(steps_total)), steps_total, alpha=0.5, color='green', width=6)
# plt.show()

In [ ]:
def UCB (Q):
    Q_init = 0
    i = 0
    Q_new_lis = []
    i_lis =[]
    for i,q in enumerate(Q):
        # print(q)
        # print('OK')
        Q_current = Q_init
        loss = 0.1 * (q - Q_current)
        Q_new = Q_current + loss
        Q_init = Q_new
        i = i+1
        
      
    
        Q_new_lis.append(Q_init)
        i_lis.append(i)
        plt.scatter(i_lis,Q_new_lis)
        
         
    return(Q_new)

In [ ]:
UCB(df.A.values)
    

In [ ]:
df.mean()

In [ ]:
import pandas as pd
import numpy as np

#create DataFrame
df = pd.DataFrame(np.random.randint(0, 100, size=(100000, 1)), columns=list('A'))

#view DataFrame
df.mean()
